In [8]:
import matplotlib.pyplot as plt 
import matplotlib as mpl
from matplotlib.dates import (YEARLY, DateFormatter,
                              rrulewrapper, RRuleLocator, drange)
import numpy as np
import os
import datetime
import pandas as pd
import xarray as xr
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from PIL import Image

root_dir='/glade/u/home/hongli/work/sharp/basins/riogrd/analysis/'
os.chdir(root_dir)
output_folder='step3_plot_hydro'
outputfile = 'hydrgraph.png'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

sim_file='/glade/u/home/hongli/work/sharp/basins/riogrd/model/route/output/all.nc'
# q_vname1 = 'KWTroutedRunoff'
q_vname2 = 'IRFroutedRunoff'
t_vname = 'time'
time_format='%Y-%m-%d'
obs_file='Rio_Grande_08220000.txt'

plot_start_date_summary=['2008-01-02']
plot_end_date_summary = ['2018-12-31']
formatter = DateFormatter('%m/%d/%y') #'%m/%d/%y','%b-%y'
dpi_value= 100

# read
print('read data')

# read simulated flow (m^3/s)
f = xr.open_dataset(sim_file)
# sim_kwt = f[q_vname1].values[:]
sim_irf = f[q_vname2].values[:]
sim_time = f[t_vname].dt.strftime(time_format).values[:]
f.close()

sim_start_str=sim_time[0]
sim_end_str=sim_time[-1]

# read observed flow (f^3/s)
obs_read = []
obs_time_read = []
with open(obs_file) as f:
    for line in f:
        line = line.strip()
        if line and line.startswith('USGS'):
            splits = line.split()
            obs_time_read.append(splits[2])
            if len(splits)>3:#flow data exists
                obs_read.append(float(splits[3])/35.315) #convert to m^3/s
            else:
                obs_read.append(np.nan)

#fill missing date corresponding flow if exist
t_day=(pd.date_range(obs_time_read[0], obs_time_read[-1], freq='D')).tolist()
t_day=list(map(pd.Timestamp.to_pydatetime, t_day))
t_day_len=len(t_day)
if len(obs_time_read) == t_day_len:
    obs = obs_read
    obs_time = obs_time_read
else:
    obs = []
    obs_time = t_day
    for i in range(t_day_len):
        if not t_day[i] in obs_time_read:
            obs.append[np.nan]            
        else:
            obs.append[obs_read[obs_time_read.index(t_day[i])]]

# extract useful observed flow
obs_start=(datetime.datetime.strptime(sim_start_str,time_format) - datetime.datetime.strptime(obs_time[0],time_format)).days
obs_len=(datetime.datetime.strptime(sim_end_str,time_format) - datetime.datetime.strptime(sim_start_str,time_format)).days+1
obs_useful = obs[obs_start:obs_start+obs_len]
obs_time_useful = obs_time[obs_start:obs_start+obs_len]
         
# plot
print('plot')
os.chdir(os.path.join(root_dir,output_folder))
fig_files = []
for m in range(len(plot_start_date_summary)):

    fig_file='image_'+str(m+1)+'.png'
    fig_files.append(fig_file)
    
    row_num=1
    col_num=1        
    fig, ax = plt.subplots(row_num,col_num)
    fig.set_figwidth(6.5*1.5*col_num) #190mm
    fig.set_figheight(6.5*0.5*row_num) #5.61 heigh/width=3/4

    plot_start_str=plot_start_date_summary[m]
    plot_end_str=plot_end_date_summary[m]

    plot_start=((datetime.datetime.strptime(plot_start_str,time_format)-datetime.datetime.strptime(sim_start_str,time_format)).days)
    plot_len=((datetime.datetime.strptime(plot_end_str,time_format)-datetime.datetime.strptime(plot_start_str,time_format)).days+1)

    plot_start_date=datetime.datetime.strptime(plot_start_str,time_format)
    t=[plot_start_date+datetime.timedelta(days=d) for d in range(0,plot_len)]
    t_plot=[mpl.dates.date2num(d) for d in t[0:plot_len]]        

    obs_plot=obs_useful[plot_start+0:plot_start+plot_len]
#         sim_kwt_plot=sim_kwt[plot_start+0:plot_start+plot_len]
    sim_irf_plot=sim_irf[plot_start+0:plot_start+plot_len]

    # ==================== calculate KGE ====================
    obs = np.asarray(obs_plot)
    sim = np.asarray(sim_irf_plot)
    sim = np.reshape(sim, np.shape(obs))
    
    obs_cal = obs[~np.isnan(obs)]
    sim_cal = sim[~np.isnan(obs)]
    
    sd_sim=np.std(sim_cal, ddof=1)
    sd_obs=np.std(obs_cal, ddof=1)
    
    m_sim=np.mean(sim_cal)
    m_obs=np.mean(obs_cal)
    
    r=(np.corrcoef(sim_cal,obs_cal))[0,1]
    relvar=float(sd_sim)/float(sd_obs)
    bias=float(m_sim)/float(m_obs)
    
    kge=1.0-np.sqrt((r-1)**2 +(relvar-1)**2 + (bias-1)**2)
    print('KGE = '+str(kge))
    # =======================================================    
    
    ax.plot_date(t_plot, obs_plot, 'k-', linewidth=1.0, markersize=0.0, alpha=0.8, label='Observation')   
#         ax.plot_date(t_plot, sim_kwt_plot, 'b--', linewidth=1.0, markersize=0.0, alpha=0.8, label='Prediction (KWT)') 
    ax.plot_date(t_plot, sim_irf_plot, 'r--', linewidth=1, markersize=0.0, alpha=0.8, label='Simulation') # (IRF)

    #axis, label, title, legend
#         ax.set_title(titles[i], fontsize='small', weight='semibold')
    ax.set_xlabel('Date', fontsize='small')
    ax.set_ylabel('Flow $(m^3/s)$', fontsize='small')
    ax.xaxis.set_major_formatter(formatter)
    ax.xaxis.set_tick_params(labelsize='small')#rotation=30,
    ax.legend(loc='best', fontsize='x-small', framealpha=0.5) #loc='upper right', 

    plt.rc('xtick',labelsize='small')
    plt.rc('ytick',labelsize='small')                
    fig.tight_layout()
    fig.savefig(fig_file, dpi=dpi_value)
    plt.close(fig)      
    del fig
del obs, sim_irf #sim_kwt, 

# save as one figure
print('save')
widths = []
heights = []
for fig_file in fig_files:
    im = Image.open(fig_file)
    widths.append(im.width)
    heights.append(im.height)

max_width = max(widths)
total_height = sum(heights)
new_im = Image.new('RGB', (max_width, total_height))

x_offset = 0
for fig_file in fig_files:
    im = Image.open(fig_file)    
    new_im.paste(im, (0,x_offset))
    x_offset += im.size[1]
new_im.save(outputfile)

for file in os.listdir(os.getcwd()):
    if file.startswith('image'):
        os.remove(file)
[os.remove(file) for file in os.getcwd() if file.startswith('image')]   

os.chdir(root_dir)


read data
plot
KGE = 0.41433946705025493
save
